In [1]:
# Importing libraries
from pyedlut import simulation_wrapper as pyedlut
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import os
from scipy.stats import norm
import scipy.stats as stats 
from scipy.ndimage import gaussian_filter
import seaborn as sns
from scipy.stats import f_oneway

## Importing functions
import sys
sys.path.insert(1, '../data')
import figure_3_functions as fun
import simulation_function as sim
import plot_spikes_function as scatter

In [2]:
## Creating plots directory
RESULTS_DIR = "../results/figures_plots"
FIGURE_3_RESULTS_DIR = os.path.join(RESULTS_DIR, 'figure_3')
os.makedirs(FIGURE_3_RESULTS_DIR, exist_ok = True)

In [3]:
## Creating default parameters dictionary

my_dict_default = {"fraction":6, 
           "pattern": 1,
           "sigma": 40, 
            "seed": 0,
            "poisson_seed": 0,
            "seed_noise": 0, 
            "noise": 0.0, 
            "mf_goc_w": 0.0,
            "grc_goc_w": 0.0,
            "num_patterns": 7,
            "duration_pattern": 0.5,
            "interval": 0.5,
            "firing_rate": 50.0, 
            "step": 10
}

# OVERVIEW

In [4]:
## Creating dictionary to initially visualize the spikes
dict_fig_overview = {
    **my_dict_default,
    "fraction": 6,
    "num_patterns": 30, 
    "mf_goc_w": 0.00,
    "grc_goc_w": 0.00,
    "duration_pattern": 0.08, 
    "interval": 0.07
}

In [ ]:
## Retrieving activity
mf_activity = fun.mf_activity_overview(dict_fig_overview)
## Running simulation
oi, ot, _, _ = sim.simulation(dict_fig_overview, mf_activity)

## Plotting spikes
fig = scatter.plot_spikes(oi, ot, dict_fig_overview, 0.0, 10*dict_fig_overview["duration_pattern"])

# STIMULI PRESENTATION

In [6]:
# Choosing patterns with similar MF active fraction
pattern_1 = 13
pattern_2 = 3
pattern_3 = 20

In [7]:
## Creating dirtionaries for each pattern without inhibition

dict_p1_without = {
    **my_dict_default,
    "pattern": 13, 
}

dict_p2_without = {
    **my_dict_default,
    "pattern": 3, 
}

dict_p3_without = {
    **my_dict_default,
    "pattern": 20, 
}

## Creating dirtionaries for each pattern with inhibition
dict_p1_with = {
    **my_dict_default,
    "pattern": 13,
    "mf_goc_w": 0.05, 
    "grc_goc_w": 0.02
}
dict_p2_with = {
    **my_dict_default,
    "pattern": 3, 
    "mf_goc_w": 0.05, 
    "grc_goc_w": 0.02
}
dict_p3_with = {
    **my_dict_default,
    "pattern": 20, 
    "mf_goc_w": 0.05, 
    "grc_goc_w": 0.02
}

In [ ]:
## Running the simulations and retrieving GrCs firing rates matrices

mf_activity = fun.mf_activity(dict_p1_without)
_, _, matrix_3, _ = sim.simulation(dict_p1_without, mf_activity)

mf_activity = fun.mf_activity(dict_p1_with)
_, _, matrix_3_inh, _ = sim.simulation(dict_p1_with, mf_activity)

mf_activity = fun.mf_activity(dict_p2_without)
_, _, matrix_2, _ = sim.simulation(dict_p2_without, mf_activity)

mf_activity = fun.mf_activity(dict_p2_with)
_, _, matrix_2_inh, _ = sim.simulation(dict_p2_with, mf_activity)

mf_activity = fun.mf_activity(dict_p3_without)
_, _, matrix_1, _ = sim.simulation(dict_p3_without, mf_activity)

mf_activity = fun.mf_activity(dict_p3_with)
_, _, matrix_1_inh, _ = sim.simulation(dict_p3_with, mf_activity)

In [9]:
## Classifying cells in three groups depending on their preference for pattern 1, 2, or 3
sum1 = np.sum(matrix_1[:,20:], axis = 1)
sum2 = np.sum(matrix_2[:,20:], axis = 1)
sum3 = np.sum(matrix_3[:,20:], axis = 1)
pref1 = []
pref2 = []
pref3 = []

for i in range(matrix_1.shape[0]):
    if sum1[i] > sum2[i]:
        if sum1[i] > sum3[i]:
            pref1.append(i)
    elif sum2[i] > sum1[i]:
        if sum2[i] > sum3[i]:
            pref2.append(i)
        else:
            pref3.append(i)

In [ ]:
# Choosing a cell to see its activity
num = 939
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2)
plt.rcParams['figure.figsize'] = (8,3.5)
plt.rcParams["savefig.facecolor"] = "white"
p1 = matrix_1[num,:]
p2 = matrix_2[num,:]
p3 = matrix_3[num,:]

p1_s = gaussian_filter(p1, sigma=1.0)
p2_s = gaussian_filter(p2, sigma=1.0)
p3_s = gaussian_filter(p3, sigma=1.0)

ax2.plot(np.arange(p1_s.shape[0]), p1_s, c = 'dodgerblue', label = 'pattern 1')
ax2.plot(np.arange(p2_s.shape[0]), p2_s, c = 'darkslateblue', label = 'pattern 2')
ax2.plot(np.arange(p3_s.shape[0]), p3_s, c = 'purple', label = 'pattern 3')

p1 = matrix_1_inh[num,:]
p2 = matrix_2_inh[num,:]
p3 = matrix_3_inh[num,:]

p1_s_inh = gaussian_filter(p1, sigma=1.0)
p2_s_inh = gaussian_filter(p2, sigma=1.0)
p3_s_inh = gaussian_filter(p3, sigma=1.0)

ax1.plot(np.arange(p1_s_inh.shape[0]), p1_s_inh, c = 'dodgerblue', label = 'Pattern 1')
ax1.plot(np.arange(p2_s_inh.shape[0]), p2_s_inh, c = 'darkslateblue', label = 'Pattern 2')
ax1.plot(np.arange(p3_s_inh.shape[0]), p3_s_inh, c = 'purple', label = 'Pattern 3')

ax1.set_ylim([-1.0, p1_s.max()])

ax1.axis("off")
ax2.axis("off")

ax1.set_title("Control", color = 'black', fontsize = 15)
ax2.set_title("DART", color = 'red', fontsize = 15)

ax1.legend(frameon=False, fontsize = 15)

ins2 = ax2.inset_axes([0.68,-0.25,0.28,0.14])
ins2.spines[['left', 'top']].set_visible(False)
ins2.set_yticks([])
ins2.set_xticks([])
ins2.set_xlabel("1 s", fontsize = 15)

fig.savefig(FIGURE_3_RESULTS_DIR + '/3_patterns.png', bbox_inches="tight", dpi = 200)

In [11]:
## Classifying cells in three groups depending on their preference for pattern 1, 2, or 3
## Now calculating "ΔF/F" with inhibition and without

mean_before = matrix_1[:,:20].mean()
matrix_1 = matrix_1/mean_before
mean_before = matrix_2[:,:20].mean()
matrix_2 = matrix_2/mean_before
mean_before = matrix_3[:,:20].mean()
matrix_3 = matrix_3/mean_before


mean_before = matrix_1_inh[:,:20].mean()
matrix_1_inh = matrix_1_inh/mean_before
mean_before = matrix_2_inh[:,:20].mean()
matrix_2_inh = matrix_2_inh/mean_before
mean_before = matrix_3_inh[:,:20].mean()
matrix_3_inh = matrix_3_inh/mean_before

In [12]:
sum1 = np.sum(matrix_1[:,20:], axis = 1)
sum2 = np.sum(matrix_2[:,20:], axis = 1)
sum3 = np.sum(matrix_3[:,20:], axis = 1)
pref1 = []
pref2 = []
pref3 = []

for i in range(matrix_1.shape[0]):
    if sum1[i] > sum2[i]:
        if sum1[i] > sum3[i]:
            pref1.append(i)
    elif sum2[i] > sum1[i]:
        if sum2[i] > sum3[i]:
            pref2.append(i)
        else:
            pref3.append(i)


In [13]:
sum1 = np.sum(matrix_1_inh[:,20:], axis = 1)
sum2 = np.sum(matrix_2_inh[:,20:], axis = 1)
sum3 = np.sum(matrix_3_inh[:,20:], axis = 1)
pref1_inh = []
pref2_inh = []
pref3_inh = []

for i in range(matrix_1_inh.shape[0]):
    if sum1[i] > sum2[i]:
        if sum1[i] > sum3[i]:
            pref1_inh.append(i)
    elif sum2[i] > sum1[i]:
        if sum2[i] > sum3[i]:
            pref2_inh.append(i)
        else:
            pref3_inh.append(i)

In [14]:
## Number of cells in each group

In [ ]:
print(len(pref1), len(pref2), len(pref3))

In [ ]:
print(len(pref1_inh), len(pref2_inh), len(pref3_inh))

In [17]:
# Dataframes for GrCs that prefer pattern 1 with and without inhibition
# Selecting only a small sample for the swarmplots
pref1_total = pref1
pref1_total_inh = pref1_inh
pref1 = random.choices(pref1, k=300)
pref1_inh = random.choices(pref1_inh, k=300)
avg1 = np.sum(matrix_1[pref1,20:], axis = 1)/matrix_1[pref1,20:].shape[1]
avg2 = np.sum(matrix_2[pref1,20:], axis = 1)/matrix_2[pref1,20:].shape[1]
avg3 = np.sum(matrix_3[pref1,20:], axis = 1)/matrix_3[pref1,20:].shape[1] 


df_1 = pd.DataFrame(avg1)
df_1['Type'] = ['P1']*avg1.shape[0]
df_1 = df_1.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1 = pd.concat([df_1, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1 = pd.concat([df_1, df])

avg1 = np.sum(matrix_1_inh[pref1_inh,20:], axis = 1)/matrix_1_inh[pref1_inh,20:].shape[1]
avg2 = np.sum(matrix_2_inh[pref1_inh,20:], axis = 1)/matrix_2_inh[pref1_inh,20:].shape[1]
avg3 = np.sum(matrix_3_inh[pref1_inh,20:], axis = 1)/matrix_3_inh[pref1_inh,20:].shape[1] 
df_1_inh = pd.DataFrame(avg1)
df_1_inh['Type'] = ['P1']*avg1.shape[0]
df_1_inh = df_1_inh.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1_inh = pd.concat([df_1_inh, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1_inh = pd.concat([df_1_inh, df])

In [ ]:

plt.rcParams["figure.figsize"] = (9,5.5)
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2)
fig.suptitle("Pattern 1 pref", fontsize = 20, y = 1.05)

ax1 = sns.swarmplot(x="Type", y="Average Firing Rate", data=df_1_inh, size = 5, color = 'dodgerblue', alpha = 1.0, ax=ax1)
ax1.spines[['right', 'top']].set_visible(False)
ax1.set_ylabel("Response (ΔF/F)", fontsize = 17)
ax1.set_title("Control", fontsize = 16, pad = 35)
df_mean_inh1 = df_1_inh.groupby('Type', sort=False)["Average Firing Rate"].mean()
#_ = [ax1.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=3) for i, y in df_mean_inh1.reset_index()["Average Firing Rate"].items()]
ax1.tick_params('both', labelsize = 15)
ax2.tick_params('both', labelsize = 15)
ax2 = sns.swarmplot(x="Type", y="Average Firing Rate", data=df_1, size = 5, color = 'dodgerblue', alpha = 1.0, ax=ax2)
ax2.spines[['right', 'top']].set_visible(False)
ax2.set_xlabel('Type', fontsize = 17)
ax2.set(ylabel=None)
ax2.set(xlabel=None)
ax1.set(xlabel=None)
ax2.set_title("DART", fontsize = 16, color = 'red', pad = 35)
df_mean1 = df_1.groupby('Type', sort=False)["Average Firing Rate"].mean()

y_max = round(ax1.get_ylim()[1])
ax1.text(0.25, y_max - 3, '****', fontsize = 15)
ax1.text(1.55, y_max - 3, '*', fontsize = 15)
ax1.text(-0.4, y_max + 2, '-------------- **** --------------', fontsize = 15)

y_max = round(ax2.get_ylim()[1])
ax2.text(0.25, y_max - 3, '****', fontsize = 15)
ax2.text(1.55, y_max - 3, '****', fontsize = 15)
ax2.text(-0.4, y_max + 2, '-------------- **** --------------', fontsize = 15)

_ = [ax1.hlines(y, i-.45, i+.45, zorder = 99, colors = 'black', linewidth=3) for i, y in df_mean_inh1.reset_index()["Average Firing Rate"].items()]
_ = [ax2.hlines(y, i-.45, i+.45, zorder = 99, colors = 'black', linewidth=3) for i, y in df_mean1.reset_index()["Average Firing Rate"].items()]

fig.savefig(FIGURE_3_RESULTS_DIR + '/pattern1_pref.png', bbox_inches="tight", dpi = 200)


In [19]:
# Dataframes for GrCs that prefer pattern 2 with and without inhibition
# Selecting only a small sample for the swarmplots

pref2_total = pref2
pref2_total_inh = pref2_inh

pref2 = random.choices(pref2, k=300)
pref2_inh = random.choices(pref2_inh, k=300)
avg1 = np.sum(matrix_1[pref2,20:], axis = 1)/matrix_1[pref2,20:].shape[1]
avg2 = np.sum(matrix_2[pref2,20:], axis = 1)/matrix_2[pref2,20:].shape[1]
avg3 = np.sum(matrix_3[pref2,20:], axis = 1)/matrix_3[pref2,20:].shape[1] 
df_2 = pd.DataFrame(avg1)
df_2['Type'] = ['P1']*avg1.shape[0]
df_2 = df_2.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2 = pd.concat([df_2, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2 = pd.concat([df_2, df])
avg1 = np.sum(matrix_1_inh[pref2_inh,20:], axis = 1)/matrix_1_inh[pref2_inh,20:].shape[1]
avg2 = np.sum(matrix_2_inh[pref2_inh,20:], axis = 1)/matrix_2_inh[pref2_inh,20:].shape[1]
avg3 = np.sum(matrix_3_inh[pref2_inh,20:], axis = 1)/matrix_3_inh[pref2_inh,20:].shape[1]
df_2_inh = pd.DataFrame(avg1)
df_2_inh['Type'] = ['P1']*avg1.shape[0]
df_2_inh = df_2_inh.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2_inh = pd.concat([df_2_inh, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2_inh = pd.concat([df_2_inh, df])

In [ ]:

plt.rcParams["figure.figsize"] = (9,5.5)
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2)
fig.suptitle("Pattern 2 pref", fontsize = 20, y = 1.05)

ax1 = sns.swarmplot(x="Type", y="Average Firing Rate", data=df_2_inh, size = 5, color = 'darkslateblue', alpha = 1.0, ax=ax1)
ax1.spines[['right', 'top']].set_visible(False)
ax1.set_ylabel("Response (ΔF/F)", fontsize = 17)
ax1.set_title("Control", fontsize = 16, pad = 35)
df_mean_inh2 = df_2_inh.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax1.hlines(y, i-.45, i+.45, zorder=99, colors = 'black', linewidth=3) for i, y in df_mean_inh2.reset_index()["Average Firing Rate"].items()]
ax1.tick_params('both', labelsize = 15)
ax2.tick_params('both', labelsize = 15)
ax2 = sns.swarmplot(x="Type", y="Average Firing Rate", data=df_2, size = 5, color = 'darkslateblue', alpha = 1.0, ax=ax2)
ax2.spines[['right', 'top']].set_visible(False)
ax2.set_xlabel('Type', fontsize = 17)
ax2.set(ylabel=None)
ax2.set(xlabel=None)
ax1.set(xlabel=None)
ax2.set_title("DART", fontsize = 16, color = 'red', pad = 35)
df_mean2 = df_2.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax2.hlines(y, i-.45, i+.45, zorder=99, colors = 'black', linewidth=3) for i, y in df_mean2.reset_index()["Average Firing Rate"].items()]

y_max = round(ax1.get_ylim()[1])
ax1.text(0.25, y_max - 3, '****', fontsize = 15)
ax1.text(1.55, y_max - 3, '****', fontsize = 15)
ax1.text(-0.4, y_max + 2, '-------------- **** --------------', fontsize = 15)
y_max = round(ax2.get_ylim()[1])
ax2.text(0.25, y_max - 3, '****', fontsize = 15)
ax2.text(1.55, y_max - 3, '****', fontsize = 15)
ax2.text(-0.4, y_max + 2, '-------------- **** --------------', fontsize = 15)



fig.savefig(FIGURE_3_RESULTS_DIR + '/pattern2_pref.png', bbox_inches="tight", dpi = 200)


In [21]:
# Dataframes for GrCs that prefer pattern 1 with and without inhibition
# Selecting only a small sample for the swarmplots

pref3_total = pref3
pref3_total_inh = pref3_inh

pref3 = random.choices(pref3, k=300)
pref3_inh = random.choices(pref3_inh, k=300)
avg1 = np.sum(matrix_1[pref3,20:], axis = 1)/matrix_1[pref3,20:].shape[1]
avg2 = np.sum(matrix_2[pref3,20:], axis = 1)/matrix_2[pref3,20:].shape[1]
avg3 = np.sum(matrix_3[pref3,20:], axis = 1)/matrix_3[pref3,20:].shape[1] 

df_3 = pd.DataFrame(avg1)
df_3['Type'] = ['P1']*avg1.shape[0]
df_3 = df_3.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3 = pd.concat([df_3, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3 = pd.concat([df_3, df])
avg1 = np.sum(matrix_1_inh[pref3_inh ,20:], axis = 1)/matrix_1_inh[pref3_inh ,20:].shape[1]
avg2 = np.sum(matrix_2_inh[pref3_inh ,20:], axis = 1)/matrix_2_inh[pref3_inh ,20:].shape[1]
avg3 = np.sum(matrix_3_inh[pref3_inh ,20:], axis = 1)/matrix_3_inh[pref3_inh ,20:].shape[1]
df_3_inh = pd.DataFrame(avg1)
df_3_inh['Type'] = ['P1']*avg1.shape[0]
df_3_inh = df_3_inh.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3_inh = pd.concat([df_3_inh, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3_inh = pd.concat([df_3_inh, df])

In [ ]:

plt.rcParams["figure.figsize"] = (9,5.5)
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2)
fig.suptitle("Pattern 3 pref", fontsize = 20, y = 1.05)

ax1 = sns.swarmplot(x="Type", y="Average Firing Rate", data=df_3_inh, size = 5, color = 'purple', alpha = 1.0, ax=ax1)
ax1.spines[['right', 'top']].set_visible(False)
ax1.set_ylabel("Response (ΔF/F)", fontsize = 17)
ax1.set_title("Control", fontsize = 16, pad = 35)
df_mean_inh3 = df_3_inh.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax1.hlines(y, i-.45, i+.45, zorder=99, colors = 'black', linewidth=3) for i, y in df_mean_inh3.reset_index()["Average Firing Rate"].items()]
ax1.tick_params('both', labelsize = 15)
ax2.tick_params('both', labelsize = 15)
ax2 = sns.swarmplot(x="Type", y="Average Firing Rate", data=df_3, size = 5, color = 'purple', alpha = 1.0, ax=ax2)
ax2.spines[['right', 'top']].set_visible(False)
ax2.set_xlabel('Type', fontsize = 17)
ax2.set(ylabel=None)
ax2.set(xlabel=None)
ax1.set(xlabel=None)
ax2.set_title("DART", fontsize = 16, color = 'red', pad = 35)
df_mean3 = df_3.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax2.hlines(y, i-.45, i+.45, zorder=99, colors = 'black', linewidth=3) for i, y in df_mean3.reset_index()["Average Firing Rate"].items()]

y_max = round(ax1.get_ylim()[1])
ax1.text(0.25, y_max - 3, '****', fontsize = 15)
ax1.text(1.55, y_max - 3, '****', fontsize = 15)
ax1.text(-0.4, y_max + 2, '-------------- **** --------------', fontsize = 15)
y_max = round(ax2.get_ylim()[1])
ax2.text(0.25, y_max - 3, '****', fontsize = 15)
ax2.text(1.55, y_max - 3, '****', fontsize = 15)
ax2.text(-0.4, y_max + 2, '-------------- **** --------------', fontsize = 15)


fig.savefig(FIGURE_3_RESULTS_DIR + '/pattern3_pref.png', bbox_inches="tight", dpi = 200)


In [23]:
## means of each set
df_mean1 = pd.DataFrame(df_mean1)
df_mean2 = pd.DataFrame(df_mean2)
df_mean3 = pd.DataFrame(df_mean3)
df_mean_inh1 = pd.DataFrame(df_mean_inh1)
df_mean_inh2 = pd.DataFrame(df_mean_inh2)
df_mean_inh3 = pd.DataFrame(df_mean_inh3)

In [24]:
pref1 = ['Pref1', 'Pref1', 'Pref1']
new_data = {'Pref': pref1}
df_mean1 = df_mean1.assign(**new_data)
pref2 = ['Pref2', 'Pref2', 'Pref2']
new_data = {'Pref': pref2}
df_mean2 = df_mean2.assign(**new_data)
pref3 = ['Pref3', 'Pref3', 'Pref3']
new_data = {'Pref': pref3}
df_mean3 = df_mean3.assign(**new_data)

In [25]:
pref1_inh = ['Pref1', 'Pref1', 'Pref1']
new_data = {'Pref': pref1_inh}
df_mean_inh1 = df_mean_inh1.assign(**new_data)
pref2_inh = ['Pref2', 'Pref2', 'Pref2']
new_data = {'Pref': pref2_inh}
df_mean_inh2 = df_mean_inh2.assign(**new_data)
pref3_inh = ['Pref3', 'Pref3', 'Pref3']
new_data = {'Pref': pref3_inh}
df_mean_inh3 = df_mean_inh3.assign(**new_data)

In [26]:
frames = [df_mean1, df_mean2, df_mean3]
result = pd.concat(frames)
result['Type'] = result.index
frames = [df_mean_inh1, df_mean_inh2, df_mean_inh3]
result_inh = pd.concat(frames)
result_inh['Type'] = result_inh.index

In [27]:
## Normalizing dataframes

In [ ]:
maximo1 = result['Average Firing Rate']['P1'].max()
maximo2 = result['Average Firing Rate']['P2'].max()
maximo3 = result['Average Firing Rate']['P3'].max()
result['Average Firing Rate']['P1'] = result['Average Firing Rate']['P1']/maximo1
result['Average Firing Rate']['P2'] = result['Average Firing Rate']['P2']/maximo2
result['Average Firing Rate']['P3'] = result['Average Firing Rate']['P3']/maximo3

In [ ]:
maximo1 = result_inh['Average Firing Rate']['P1'].max()
maximo2 = result_inh['Average Firing Rate']['P2'].max()
maximo3 = result_inh['Average Firing Rate']['P3'].max()
result_inh['Average Firing Rate']['P1'] = result_inh['Average Firing Rate']['P1']/maximo1
result_inh['Average Firing Rate']['P2'] = result_inh['Average Firing Rate']['P2']/maximo2
result_inh['Average Firing Rate']['P3'] = result_inh['Average Firing Rate']['P3']/maximo3

In [ ]:
prefs = ['Pref1', 'Pref2', 'Pref3']
colors = ['dodgerblue', 'darkslateblue', 'purple']
plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

for i in range(len(prefs)):
    
    df = result_inh[result_inh['Pref'] == prefs[i]]
    ax.scatter(df['Type'].values, df['Average Firing Rate'].values, c = colors[i], s = 100)
    ax.plot(df['Type'].values, df['Average Firing Rate'].values, c = colors[i])
    
ax.tick_params('both', labelsize = 15)
ax.set_ylim([0, 1.1])
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
ax.set_title('Control', fontsize = 17)
#fig.savefig(path + '/control_response_all.png', bbox_inches="tight", dpi = 200)

In [ ]:

plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

df = result[result['Pref'] == prefs[0]] 
df_inh = result_inh[result_inh['Pref'] == prefs[0]]
ax.scatter(df['Type'].values, df['Average Firing Rate'].values, c = 'red', s = 100)
ax.plot(df['Type'].values, df['Average Firing Rate'].values, c = 'red')
ax.scatter(df_inh['Type'].values, df_inh['Average Firing Rate'].values, c = colors[0], s = 100)
ax.plot(df_inh['Type'].values, df_inh['Average Firing Rate'].values, c = colors[0])

ax.tick_params('both', labelsize = 15)
ax.set_ylim([0, 1.1])
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
ax.set_title('DART', color = 'red', fontsize = 15)
#fig.savefig(path + '/dart_response_1.png', bbox_inches="tight", dpi = 200)

In [ ]:
prefs = ['Pref1', 'Pref2', 'Pref3']

plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

df = result[result['Pref'] == prefs[1]] 
df_inh = result_inh[result_inh['Pref'] == prefs[1]]
ax.scatter(df['Type'].values, df['Average Firing Rate'].values, c = 'red', s = 100)
ax.plot(df['Type'].values, df['Average Firing Rate'].values, c = 'red')
ax.scatter(df_inh['Type'].values, df_inh['Average Firing Rate'].values, c = colors[1], s = 100)
ax.plot(df_inh['Type'].values, df_inh['Average Firing Rate'].values, c = colors[1])

ax.tick_params('both', labelsize = 15)
ax.set_ylim([0, 1.1])
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
#fig.savefig(path + '/dart_response_2.png', bbox_inches="tight", dpi = 200)

In [ ]:
prefs = ['Pref1', 'Pref2', 'Pref3']

plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

df = result[result['Pref'] == prefs[2]] 
df_inh = result_inh[result_inh['Pref'] == prefs[2]]
ax.scatter(df['Type'].values, df['Average Firing Rate'].values, c = 'red', s = 100)
ax.plot(df['Type'].values, df['Average Firing Rate'].values, c = 'red')
ax.scatter(df_inh['Type'].values, df_inh['Average Firing Rate'].values, c = colors[2], s = 100)
ax.plot(df_inh['Type'].values, df_inh['Average Firing Rate'].values, c = colors[2])
ax.set_ylim([0, 1.1])
ax.tick_params('both', labelsize = 15)
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
#fig.savefig(path + '/dart_response_3.png', bbox_inches="tight", dpi = 200)

# VIOLIN PLOTS VERSIONS THAT INCLUDE ALL THE CELLS AND STATISTICS 

In [34]:
# Dataframes for GrCs that prefer pattern 1 with and without inhibition
# Violinplots

avg1 = np.sum(matrix_1[pref1_total,20:], axis = 1)/matrix_1[pref1_total,20:].shape[1]
avg2 = np.sum(matrix_2[pref1_total,20:], axis = 1)/matrix_2[pref1_total,20:].shape[1]
avg3 = np.sum(matrix_3[pref1_total,20:], axis = 1)/matrix_3[pref1_total,20:].shape[1] 
df_1 = pd.DataFrame(avg1)
df_1['Type'] = ['P1']*avg1.shape[0]
df_1 = df_1.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1 = pd.concat([df_1, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1 = pd.concat([df_1, df])
avg1 = np.sum(matrix_1_inh[pref1_total_inh,20:], axis = 1)/matrix_1_inh[pref1_total_inh,20:].shape[1]
avg2 = np.sum(matrix_2_inh[pref1_total_inh,20:], axis = 1)/matrix_2_inh[pref1_total_inh,20:].shape[1]
avg3 = np.sum(matrix_3_inh[pref1_total_inh,20:], axis = 1)/matrix_3_inh[pref1_total_inh,20:].shape[1] 
df_1_inh = pd.DataFrame(avg1)
df_1_inh['Type'] = ['P1']*avg1.shape[0]
df_1_inh = df_1_inh.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1_inh = pd.concat([df_1_inh, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_1_inh = pd.concat([df_1_inh, df])

In [ ]:

plt.rcParams["figure.figsize"] = (8,5)
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2)
fig.suptitle("Pattern 1 pref", fontsize = 20, y = 1.10)

ax1 = sns.violinplot(x="Type", y="Average Firing Rate", data=df_1_inh, color = 'dodgerblue', alpha = 1.0, ax=ax1)
ax1.spines[['right', 'top']].set_visible(False)
ax1.set_ylabel("Response (ΔF/F)", fontsize = 17)
ax1.set_title("Control", fontsize = 16, pad = 35)
df_mean_inh1 = df_1_inh.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax1.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=3) for i, y in df_mean_inh1.reset_index()["Average Firing Rate"].items()]
ax1.tick_params('both', labelsize = 15)
ax2.tick_params('both', labelsize = 15)
ax2 = sns.violinplot(x="Type", y="Average Firing Rate", data=df_1, color = 'dodgerblue', alpha = 1.0, ax=ax2)
ax2.spines[['right', 'top']].set_visible(False)
ax2.set_xlabel('Type', fontsize = 17)
ax2.set(ylabel=None)
ax2.set(xlabel=None)
ax1.set(xlabel=None)
ax2.set_title("DART", fontsize = 16, color = 'red', pad = 35)
df_mean1 = df_1.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax2.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=3) for i, y in df_mean1.reset_index()["Average Firing Rate"].items()]


y_max = round(ax1.get_ylim()[1])
ax1.text(0.25, y_max - 6, '****', fontsize = 15)
ax1.text(1.55, y_max - 6, 'NS', fontsize = 15)
ax1.text(-0.1, y_max , '--------- **** ---------', fontsize = 15)
y_max = round(ax2.get_ylim()[1])
ax2.text(0.25, y_max - 6, '****', fontsize = 15)
ax2.text(1.35, y_max - 6, '****', fontsize = 15)
ax2.text(-0.1, y_max , '--------- **** ---------', fontsize = 15)

fig.savefig(FIGURE_3_RESULTS_DIR + '/violin_pattern1_pref.png', bbox_inches="tight", dpi = 200)

In [36]:

## Hypothesis tests
## Notice that P-values results were calculated and then they were manually written in every plot
## Therefore they should be edited for each combination of parameters computed
test12 = stats.ttest_rel(df_1[df_1["Type"] == "P1"]["Average Firing Rate"], 
         df_1[df_1["Type"] == "P2"]["Average Firing Rate"])
test13 = stats.ttest_rel(df_1[df_1["Type"] == "P1"]["Average Firing Rate"], 
         df_1[df_1["Type"] == "P3"]["Average Firing Rate"])   
test23 = stats.ttest_rel(df_1[df_1["Type"] == "P2"]["Average Firing Rate"], 
         df_1[df_1["Type"] == "P3"]["Average Firing Rate"])   

In [ ]:
print(test12)
print(test13)
print(test23)

In [38]:
test12 = stats.ttest_rel(df_1_inh[df_1_inh["Type"] == "P1"]["Average Firing Rate"], 
         df_1_inh[df_1_inh["Type"] == "P2"]["Average Firing Rate"])
test13 = stats.ttest_rel(df_1_inh[df_1_inh["Type"] == "P1"]["Average Firing Rate"], 
         df_1_inh[df_1_inh["Type"] == "P3"]["Average Firing Rate"])   
test23 = stats.ttest_rel(df_1_inh[df_1_inh["Type"] == "P2"]["Average Firing Rate"], 
         df_1_inh[df_1_inh["Type"] == "P3"]["Average Firing Rate"])   

In [ ]:
print(test12)
print(test13)
print(test23)

In [ ]:
df_mean1

In [ ]:
df_1.groupby('Type', sort=False)["Average Firing Rate"].sem()

In [ ]:
df_mean_inh1

In [ ]:
df_1_inh.groupby('Type', sort=False)["Average Firing Rate"].sem()

In [44]:
# Dataframes for GrCs that prefer pattern 2 with and without inhibition
# Violinplots

pref = pref2_total
avg1 = np.sum(matrix_1[pref2_total,20:], axis = 1)/matrix_1[pref2_total,20:].shape[1]
avg2 = np.sum(matrix_2[pref2_total,20:], axis = 1)/matrix_2[pref2_total,20:].shape[1]
avg3 = np.sum(matrix_3[pref2_total,20:], axis = 1)/matrix_3[pref2_total,20:].shape[1] 
df_2 = pd.DataFrame(avg1)
df_2['Type'] = ['P1']*avg1.shape[0]
df_2 = df_2.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2 = pd.concat([df_2, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2 = pd.concat([df_2, df])
avg1 = np.sum(matrix_1_inh[pref2_total_inh,20:], axis = 1)/matrix_1_inh[pref2_total_inh,20:].shape[1]
avg2 = np.sum(matrix_2_inh[pref2_total_inh,20:], axis = 1)/matrix_2_inh[pref2_total_inh,20:].shape[1]
avg3 = np.sum(matrix_3_inh[pref2_total_inh,20:], axis = 1)/matrix_3_inh[pref2_total_inh,20:].shape[1]
df_2_inh = pd.DataFrame(avg1)
df_2_inh['Type'] = ['P1']*avg1.shape[0]
df_2_inh = df_2_inh.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2_inh = pd.concat([df_2_inh, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_2_inh = pd.concat([df_2_inh, df])

In [ ]:

plt.rcParams["figure.figsize"] = (8,5)
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2)
fig.suptitle("Pattern 2 pref", fontsize = 20, y =1.10)

ax1 = sns.violinplot(x="Type", y="Average Firing Rate", data=df_2_inh, color = 'darkslateblue', alpha = 1.0, ax=ax1)
ax1.spines[['right', 'top']].set_visible(False)
ax1.set_ylabel("Response (ΔF/F)", fontsize = 17)
ax1.set_title("Control", fontsize = 16, pad = 35)
df_mean_inh2 = df_2_inh.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax1.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=3) for i, y in df_mean_inh2.reset_index()["Average Firing Rate"].items()]
ax1.tick_params('both', labelsize = 15)
ax2.tick_params('both', labelsize = 15)
ax2 = sns.violinplot(x="Type", y="Average Firing Rate", data=df_2, color = 'darkslateblue', alpha = 1.0, ax=ax2)
ax2.spines[['right', 'top']].set_visible(False)
ax2.set_xlabel('Type', fontsize = 17)
ax2.set(ylabel=None)
ax2.set(xlabel=None)
ax1.set(xlabel=None)
ax2.set_title("DART", fontsize = 16, color = 'red', pad = 35)
df_mean2 = df_2.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax2.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=3) for i, y in df_mean2.reset_index()["Average Firing Rate"].items()]

y_max = round(ax1.get_ylim()[1])
ax1.text(0.25, y_max - 6, '****', fontsize = 15)
ax1.text(1.35, y_max - 6, '****', fontsize = 15)
ax1.text(-0.1, y_max , '--------- **** ---------', fontsize = 15)
y_max = round(ax2.get_ylim()[1])
ax2.text(0.25, y_max - 6, '****', fontsize = 15)
ax2.text(1.35, y_max - 6, '****', fontsize = 15)
ax2.text(-0.1, y_max , '--------- **** ---------', fontsize = 15)
fig.savefig(FIGURE_3_RESULTS_DIR + '/violin_pattern2_pref.png', bbox_inches="tight", dpi = 200)


In [ ]:

test12 = stats.ttest_rel(df_2[df_2["Type"] == "P1"]["Average Firing Rate"], 
         df_2[df_2["Type"] == "P2"]["Average Firing Rate"])
test13 = stats.ttest_rel(df_2[df_2["Type"] == "P1"]["Average Firing Rate"], 
         df_2[df_2["Type"] == "P3"]["Average Firing Rate"])   
test23 = stats.ttest_rel(df_2[df_2["Type"] == "P2"]["Average Firing Rate"], 
         df_2[df_2["Type"] == "P3"]["Average Firing Rate"])   
print(test12)
print(test13)
print(test23)

In [ ]:
test12 = stats.ttest_rel(df_2_inh[df_2_inh["Type"] == "P1"]["Average Firing Rate"], 
         df_2_inh[df_2_inh["Type"] == "P2"]["Average Firing Rate"])
test13 = stats.ttest_rel(df_2_inh[df_2_inh["Type"] == "P1"]["Average Firing Rate"], 
         df_2_inh[df_2_inh["Type"] == "P3"]["Average Firing Rate"])   
test23 = stats.ttest_rel(df_2_inh[df_2_inh["Type"] == "P2"]["Average Firing Rate"], 
         df_2_inh[df_2_inh["Type"] == "P3"]["Average Firing Rate"])   
print(test12)
print(test13)
print(test23)

In [ ]:
df_mean2

In [ ]:
df_2.groupby('Type', sort=False)["Average Firing Rate"].sem()

In [ ]:
df_mean_inh2

In [ ]:
df_2_inh.groupby('Type', sort=False)["Average Firing Rate"].sem()

In [52]:
# Dataframes for GrCs that prefer pattern 3 with and without inhibition
# Violinplots


pref = pref3_total
avg1 = np.sum(matrix_1[pref3_total,20:], axis = 1)/matrix_1[pref3_total,20:].shape[1]
avg2 = np.sum(matrix_2[pref3_total,20:], axis = 1)/matrix_2[pref3_total,20:].shape[1]
avg3 = np.sum(matrix_3[pref3_total,20:], axis = 1)/matrix_3[pref3_total,20:].shape[1] 

df_3 = pd.DataFrame(avg1)
df_3['Type'] = ['P1']*avg1.shape[0]
df_3 = df_3.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3 = pd.concat([df_3, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3 = pd.concat([df_3, df])
avg1 = np.sum(matrix_1_inh[pref3_total_inh,20:], axis = 1)/matrix_1_inh[pref3_total_inh,20:].shape[1]
avg2 = np.sum(matrix_2_inh[pref3_total_inh,20:], axis = 1)/matrix_2_inh[pref3_total_inh,20:].shape[1]
avg3 = np.sum(matrix_3_inh[pref3_total_inh,20:], axis = 1)/matrix_3_inh[pref3_total_inh,20:].shape[1]
df_3_inh = pd.DataFrame(avg1)
df_3_inh['Type'] = ['P1']*avg1.shape[0]
df_3_inh = df_3_inh.rename(columns={0: 'Average Firing Rate'})
df = pd.DataFrame(avg2)
df['Type'] = ['P2']*avg2.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3_inh = pd.concat([df_3_inh, df])
df = pd.DataFrame(avg3)
df['Type'] = ['P3']*avg3.shape[0]
df = df.rename(columns={0: 'Average Firing Rate'})
df_3_inh = pd.concat([df_3_inh, df])

In [ ]:

plt.rcParams["figure.figsize"] = (8,5)
fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2)
fig.suptitle("Pattern 3 pref", fontsize = 20, y = 1.10)

ax1 = sns.violinplot(x="Type", y="Average Firing Rate", data=df_3_inh, color = 'purple', alpha = 1.0, ax=ax1)
ax1.spines[['right', 'top']].set_visible(False)
ax1.set_ylabel("Response (ΔF/F)", fontsize = 17)
ax1.set_title("Control", fontsize = 16, pad = 35)
df_mean_inh3 = df_3_inh.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax1.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=3) for i, y in df_mean_inh3.reset_index()["Average Firing Rate"].items()]
ax1.tick_params('both', labelsize = 15)
ax2.tick_params('both', labelsize = 15)
ax2 = sns.violinplot(x="Type", y="Average Firing Rate", data=df_3, color = 'purple', alpha = 1.0, ax=ax2)
ax2.spines[['right', 'top']].set_visible(False)
ax2.set_xlabel('Type', fontsize = 17)
ax2.set(ylabel=None)
ax2.set(xlabel=None)
ax1.set(xlabel=None)
ax2.set_title("DART", fontsize = 16, color = 'red', pad = 35)
df_mean3 = df_3.groupby('Type', sort=False)["Average Firing Rate"].mean()
_ = [ax2.hlines(y, i-.45, i+.45, zorder=2, colors = 'black', linewidth=3) for i, y in df_mean3.reset_index()["Average Firing Rate"].items()]


y_max = round(ax1.get_ylim()[1])
ax1.text(0.25, y_max - 6, '****', fontsize = 15)
ax1.text(1.35, y_max - 6, '****', fontsize = 15)
ax1.text(-0.1, y_max , '--------- **** ---------', fontsize = 15)
y_max = round(ax2.get_ylim()[1])
ax2.text(0.25, y_max - 6, '****', fontsize = 15)
ax2.text(1.35, y_max - 6, '****', fontsize = 15)
ax2.text(-0.1, y_max , '--------- **** ---------', fontsize = 15)

fig.savefig(FIGURE_3_RESULTS_DIR + '/violin_pattern3_pref.png', bbox_inches="tight", dpi = 200)


In [ ]:

test12 = stats.ttest_rel(df_3[df_3["Type"] == "P1"]["Average Firing Rate"], 
         df_3[df_3["Type"] == "P2"]["Average Firing Rate"])
test13 = stats.ttest_rel(df_3[df_3["Type"] == "P1"]["Average Firing Rate"], 
         df_3[df_3["Type"] == "P3"]["Average Firing Rate"])   
test23 = stats.ttest_rel(df_3[df_3["Type"] == "P2"]["Average Firing Rate"], 
         df_3[df_3["Type"] == "P3"]["Average Firing Rate"])   
print(test12)
print(test13)
print(test23)

In [ ]:
test12 = stats.ttest_rel(df_3_inh[df_3_inh["Type"] == "P1"]["Average Firing Rate"], 
         df_3_inh[df_3_inh["Type"] == "P2"]["Average Firing Rate"])
test13 = stats.ttest_rel(df_3_inh[df_3_inh["Type"] == "P1"]["Average Firing Rate"], 
         df_3_inh[df_3_inh["Type"] == "P3"]["Average Firing Rate"])   
test23 = stats.ttest_rel(df_3_inh[df_3_inh["Type"] == "P2"]["Average Firing Rate"], 
         df_3_inh[df_3_inh["Type"] == "P3"]["Average Firing Rate"])   
print(test12)
print(test13)
print(test23)

In [ ]:
df_mean3

In [ ]:
df_3.groupby('Type', sort=False)["Average Firing Rate"].sem()

In [ ]:
df_mean_inh3

In [ ]:
df_3_inh.groupby('Type', sort=False)["Average Firing Rate"].sem()

In [60]:
## pattern 1 preferred 
df_1_inh_norm = df_1_inh
df_1_inh_norm["Average Firing Rate"] =  df_1_inh["Average Firing Rate"]/df_1_inh["Average Firing Rate"].max()
df_2_inh_norm = df_2_inh
df_2_inh_norm["Average Firing Rate"] =  df_2_inh["Average Firing Rate"]/df_2_inh["Average Firing Rate"].max()
df_3_inh_norm = df_3_inh
df_3_inh_norm["Average Firing Rate"] =  df_3_inh["Average Firing Rate"]/df_3_inh["Average Firing Rate"].max()

In [61]:
pref1 = ['Pref1']*len(df_1_inh_norm)
new_data = {'Pref': pref1}
df_1_inh_norm = df_1_inh_norm.assign(**new_data)
pref2 = ['Pref2']*len(df_2_inh_norm)
new_data = {'Pref': pref2}
df_2_inh_norm = df_2_inh_norm.assign(**new_data)
pref3 = ['Pref3']*len(df_3_inh_norm)
new_data = {'Pref': pref3}
df_3_inh_norm = df_3_inh_norm.assign(**new_data)

In [62]:
frames = [df_1_inh_norm, df_2_inh_norm, df_3_inh_norm]
result_inh = pd.concat(frames)

In [63]:
## pattern 1 preferred 
df_1_norm = df_1
df_1_norm["Average Firing Rate"] =  df_1["Average Firing Rate"]/df_1["Average Firing Rate"].max()
df_2_norm = df_2
df_2_norm["Average Firing Rate"] =  df_2["Average Firing Rate"]/df_2["Average Firing Rate"].max()
df_3_norm = df_3
df_3_norm["Average Firing Rate"] =  df_3["Average Firing Rate"]/df_3["Average Firing Rate"].max()

In [64]:
pref1 = ['Pref1']*len(df_1_norm)
new_data = {'Pref': pref1}
df_1_norm = df_1_norm.assign(**new_data)
pref2 = ['Pref2']*len(df_2_norm)
new_data = {'Pref': pref2}
df_2_norm = df_2_norm.assign(**new_data)
pref3 = ['Pref3']*len(df_3_norm)
new_data = {'Pref': pref3}
df_3_norm = df_3_norm.assign(**new_data)

In [65]:
frames = [df_1_norm, df_2_norm, df_3_norm]
result = pd.concat(frames)

In [ ]:
prefs = ['Pref1', 'Pref2', 'Pref3']
colors = ['dodgerblue', 'darkslateblue', 'purple']
plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

for i in range(len(prefs)):
    
    df = result_inh[result_inh['Pref'] == prefs[i]]
    df_new = df.groupby(by = ["Type"])['Average Firing Rate'].mean()
    df_sem = df.groupby(by = ["Type"])['Average Firing Rate'].sem()
    
    ax.scatter(df_new.index.values, df_new.values/df_new.values.max(), c = colors[i], s = 80)
    ax.plot(df_new.index.values, df_new.values/df_new.values.max(), c = colors[i])
    ax.errorbar(df_new.index.values, df_new.values/df_new.values.max(), df_sem.values, c = colors[i])
    
ax.tick_params('both', labelsize = 15)
ax.set_ylim([0, 1.1])
ax.margins(x=0.15)
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
ax.set_title('Control', fontsize = 17)
fig.savefig(FIGURE_3_RESULTS_DIR + '/control_response_all.png', bbox_inches="tight", dpi = 200)


In [ ]:

plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

df = result[result['Pref'] == prefs[0]] 
df_new = df.groupby(by = ["Type"])['Average Firing Rate'].mean()
df_sem = df.groupby(by = ["Type"])['Average Firing Rate'].sem()

df_inh = result_inh[result_inh['Pref'] == prefs[0]]
df_inh_new = df_inh.groupby(by = ["Type"])['Average Firing Rate'].mean()
df_inh_sem = df_inh.groupby(by = ["Type"])['Average Firing Rate'].sem()

ax.scatter(df_new.index.values, df_new.values/df_new.values.max(), c = 'red', s = 100)
ax.plot(df_new.index.values, df_new.values/df_new.values.max(), c = 'red')
ax.errorbar(df_new.index.values, df_new.values/df_new.values.max(), df_sem.values, c = 'red')

ax.scatter(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), c = colors[0], s = 100)
ax.plot(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), c = colors[0])
ax.errorbar(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), df_inh_sem.values, c = colors[0])


ax.tick_params('both', labelsize = 15)
ax.set_ylim([0, 1.1])
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
ax.set_title('DART', color = 'red', fontsize = 15)
y_max = round(ax.get_ylim()[1])
ax.margins(x=0.15)
ax.text(0.80, y_max, 'NS', fontsize = 15)
ax.text(1.85, y_max, '****', fontsize = 15)
fig.savefig(FIGURE_3_RESULTS_DIR + '/dart_response_1.png', bbox_inches="tight", dpi = 200)


In [ ]:
stats.ttest_ind(df[df["Type"] == "P2"]["Average Firing Rate"], 
         df_inh[df_inh["Type"] == "P2"]["Average Firing Rate"])

In [ ]:
stats.ttest_ind(df[df["Type"] == "P3"]["Average Firing Rate"], 
         df_inh[df_inh["Type"] == "P3"]["Average Firing Rate"])

In [ ]:

f_oneway(df[df["Type"] == "P2"]["Average Firing Rate"], df_inh[df_inh["Type"] == "P2"]["Average Firing Rate"])

In [ ]:
f_oneway(df[df["Type"] == "P3"]["Average Firing Rate"], df_inh[df_inh["Type"] == "P3"]["Average Firing Rate"])

In [ ]:
plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

df = result[result['Pref'] == prefs[1]] 
df_new = df.groupby(by = ["Type"])['Average Firing Rate'].mean()
df_sem = df.groupby(by = ["Type"])['Average Firing Rate'].sem()

df_inh = result_inh[result_inh['Pref'] == prefs[1]]
df_inh_new = df_inh.groupby(by = ["Type"])['Average Firing Rate'].mean()
df_inh_sem = df_inh.groupby(by = ["Type"])['Average Firing Rate'].sem()

ax.scatter(df_new.index.values, df_new.values/df_new.values.max(), c = 'red', s = 100)
ax.plot(df_new.index.values, df_new.values/df_new.values.max(), c = 'red')
ax.errorbar(df_new.index.values, df_new.values/df_new.values.max(), df_sem.values, c = 'red')

ax.scatter(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), c = colors[1], s = 100)
ax.plot(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), c = colors[1])
ax.errorbar(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), df_inh_sem.values, c = colors[1])

ax.tick_params('both', labelsize = 15)
ax.set_ylim([0, 1.1])
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
ax.set_title('DART', color = 'red', fontsize = 15)
ax.margins(x=0.15)
y_max = round(ax.get_ylim()[1])
ax.text(-0.05, y_max, '****', fontsize = 15)
ax.text(1.85, y_max, 'NS', fontsize = 15)
fig.savefig(FIGURE_3_RESULTS_DIR + '/dart_response_2.png', bbox_inches="tight", dpi = 200)


In [73]:
stats.ttest_rel(df[df["Type"] == "P1"]["Average Firing Rate"], 
          df_inh[df_inh["Type"] == "P1"]["Average Firing Rate"])

In [74]:
stats.ttest_rel(df[df["Type"] == "P3"]["Average Firing Rate"], 
         df_inh[df_inh["Type"] == "P3"]["Average Firing Rate"])

In [ ]:
f_oneway(df[df["Type"] == "P1"]["Average Firing Rate"], df_inh[df_inh["Type"] == "P1"]["Average Firing Rate"])

In [ ]:
f_oneway(df[df["Type"] == "P3"]["Average Firing Rate"], df_inh[df_inh["Type"] == "P3"]["Average Firing Rate"])

In [ ]:
plt.rcParams["figure.figsize"] = (3.5,5)
fig, ax = plt.subplots()

df = result[result['Pref'] == prefs[2]] 
df_new = df.groupby(by = ["Type"])['Average Firing Rate'].mean()
df_sem = df.groupby(by = ["Type"])['Average Firing Rate'].sem()

df_inh = result_inh[result_inh['Pref'] == prefs[2]]
df_inh_new = df_inh.groupby(by = ["Type"])['Average Firing Rate'].mean()
df_inh_sem = df_inh.groupby(by = ["Type"])['Average Firing Rate'].sem()

ax.scatter(df_new.index.values, df_new.values/df_new.values.max(), c = 'red', s = 100)
ax.plot(df_new.index.values, df_new.values/df_new.values.max(), c = 'red')
ax.errorbar(df_new.index.values, df_new.values/df_new.values.max(), df_sem.values, c = 'red')

ax.scatter(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), c = colors[2], s = 100, facecolors='none')
ax.plot(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), c = colors[2])
ax.errorbar(df_inh_new.index.values, df_inh_new.values/df_inh_new.values.max(), df_inh_sem.values, c = colors[2])

ax.tick_params('both', labelsize = 15)
ax.set_ylim([0, 1.1])
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylabel('Response (norm.)', fontsize = 15)
ax.set_title('DART', color = 'red', fontsize = 15)
ax.margins(x=0.15)
ax.text(0.0, y_max, 'NS', fontsize = 15)
ax.text(0.80, y_max, '***', fontsize = 15)
fig.savefig(FIGURE_3_RESULTS_DIR + '/dart_response_3.png', bbox_inches="tight", dpi = 200)

In [ ]:

f_oneway(df[df["Type"] == "P1"]["Average Firing Rate"], df_inh[df_inh["Type"] == "P1"]["Average Firing Rate"])

In [ ]:
f_oneway(df[df["Type"] == "P2"]["Average Firing Rate"], df_inh[df_inh["Type"] == "P2"]["Average Firing Rate"])